In [1]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
modelLM = SentenceTransformer("all-MiniLM-L6-v2")

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
modelUSE = hub.load(module_url)

In [10]:
from qtn_sim import *


circuit1 = QCircuit()
circuit1.addGate(HGate(), [0])
circuit1.addGate(CNOTGate(), [0,1])
circuit1.addGate(XGate(), [1])

circuit2 = QCircuit()
circuit2.addGate(HGate(), [0])

circuit3 = QCircuit()
circuit3.addGate(CNOTGate(), [0,1])
circuit3.addGate(XGate(), [1])

circuit4 = QCircuit()
circuit4.addGate(TOFFOLIGate(), [0,1,2])

circuits = [circuit1, circuit2, circuit3, circuit4]

In [13]:
c = [modelUSE([circ.toString()]).numpy()[0] for circ in circuits]

from sklearn.metrics.pairwise import pairwise_distances

print("Pairwise dist")
print(pairwise_distances(c, metric='sqeuclidean'))


from sklearn.decomposition import PCA
pca = PCA(n_components=3)
x = pca.fit_transform(c)

print(x)

print("PCA 3 components")
print(x)

print("First circuit")
print(x[0])
print("Sum of both c1 and c2")
print(x[1]+ x[2])

print("Dist from first circuit to sum of 2nd and 3rd ")
print(np.linalg.norm(x[0] - (x[1]+x[2])))
print(np.linalg.norm(x[0] - x[3]))

x = c

Pairwise dist
[[0.         0.70889982 0.85483866 1.30441326]
 [0.70889982 0.         1.8566337  1.95847407]
 [0.85483866 1.8566337  0.         1.06119021]
 [1.30441326 1.95847407 1.06119021 0.        ]]
[[-0.25565307 -0.19607227  0.35945376]
 [-0.75750262  0.17794213 -0.20371592]
 [ 0.4307547  -0.4886776  -0.18649968]
 [ 0.58240099  0.50680773  0.03076184]]
PCA 3 components
[[-0.25565307 -0.19607227  0.35945376]
 [-0.75750262  0.17794213 -0.20371592]
 [ 0.4307547  -0.4886776  -0.18649968]
 [ 0.58240099  0.50680773  0.03076184]]
First circuit
[-0.25565307 -0.19607227  0.35945376]
Sum of both c1 and c2
[-0.32674792 -0.31073547 -0.3902156 ]
Dist from first circuit to sum of 2nd and 3rd 
0.7617127244924289
1.1421091284035807


In [12]:
# print(x)

print(x[0][:2])
print(x[1][:2] + x[2][:2])

print(np.linalg.norm(x[0].numpy() - (x[1].numpy()+x[2].numpy())))
print(np.linalg.norm(x[0].numpy() - x[3].numpy()))


[-0.0007877 -0.0558197]
[-0.03287665 -0.10648857]


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [ ]:
# Checking permutation invariance. 
from qtn_sim import *
# circuit1 = QCircuit()
# circuit1.addGate(HGate(), [0])
# circuit1.addGate(CNOTGate(), [2,5])
# circuit1.addGate(CNOTGate(), [0,1])


# circuit2 = QCircuit()
# # circuit2.addGate(HGate(), [0])
# # circuit2.addGate(CNOTGate(), [0,1])
# # circuit2.addGate(CNOTGate(), [2,5])

# circuits = [circuit1, circuit2]

# circuits = createCircuits(numberOfCircuits=5000, 
#                           numberOfQubits=10, 
#                           maxCircuitDepth=10, 
#                           withNonAdjacent=True, 
#                           filePath="circuits.json")

circuits = readCircuits("circuits.json")


c = np.array([modelUSE([circ.toString()]).numpy()[0] for circ in circuits])

from sklearn.metrics.pairwise import pairwise_distances

print("Pairwise dist")
print(pairwise_distances(c, metric='sqeuclidean'))

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
x = pca.fit_transform(c)

print("PCA 2 components")
print(x)

def getSparsity(v):
    return np.count_nonzero(v)


y = []

for C in circuits:
    q = QuantumMPS(10)
    q.applyCircuit(C)
    y.append(getSparsity(q.get_state_vector()))


import pandas as pd 




Pairwise dist
[[0.         1.36726642 0.89443051 ... 2.07523242 1.59255434 1.29634497]
 [1.36726642 0.         1.61552905 ... 1.67277451 1.58985879 1.37486895]
 [0.89443051 1.61552905 0.         ... 1.96871496 1.77511757 1.29498983]
 ...
 [2.07523242 1.67277451 1.96871496 ... 0.         1.48843763 2.02022882]
 [1.59255434 1.58985879 1.77511757 ... 1.48843763 0.         1.5217839 ]
 [1.29634497 1.37486895 1.29498983 ... 2.02022882 1.5217839  0.        ]]
PCA 2 components
[[-0.33973208  0.08240659]
 [ 0.23265597  0.4120434 ]
 [-0.23746203 -0.31562203]
 ...
 [ 0.49611095  0.00302926]
 [ 0.42138585  0.05581953]
 [ 0.23588225 -0.01091935]]


ValueError: Per-column arrays must each be 1-dimensional

In [77]:
data = {
    'pc1': x[:,0],
    'pc2': x[:,1],
    'sparsity': y
}
# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('df.csv', index=False)